In [163]:
#Import Dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [164]:
#Load in data from csv
pipes_df = pd.read_csv("Pipe_data.csv")
pipes_df.head()

C:\Users\shwet\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (57,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,FID,MOID,DIAMETER,PIPE_LENGTH,MATERIAL,DESCRIPTION,YEAR_INSTALLED,PRESSURE_ZONE,LINING,LINE_YEAR,...,WORK_ORDER_NUMBER,Comments,ReportDate,StartDateT,CompletedD,ReportedY,SrvcRqst_1,MATERIAL_1,MUSYM,FAILURE
0,16420,P35881,8.0,861.090481,M.J.,Distribution,1952,Hemp 1175,,0,...,1143689,PER UNIT 819\n\rREQUEST 1 6X12 PLATE AND 4 BAG...,2014-10-25,2014-11-16,2014-11-16,2014,Water Request,CAS,Ub,1
1,44465,P88079,8.0,749.511184,TYT,Distribution,1965,Hemp 1175,,0,...,1311148,MAIN BREAK,2015-08-18,2015-08-18,2015-08-18,2015,Water Request,DIP,ReD,1
2,23810,P53764,8.0,171.554040,M.J.,Distribution,1958,Hemp 1175,,0,...,1323748,PER JONATHAN WEBB THERE IS A MAIN BREAK AT THI...,2015-09-11,2015-09-12,2015-09-12,2015,Water Pressure Complaint,DIP,Ub,1
3,52630,P400741,8.0,42.761097,M.J.,Distribution,1958,Hemp 1175,,0,...,1323748,PER JONATHAN WEBB THERE IS A MAIN BREAK AT THI...,2015-09-11,2015-09-12,2015-09-12,2015,Water Pressure Complaint,DIP,Ub,1
4,10510,P20210,12.0,31.300481,DIP,Transmission,2006,Hemp 1175,,0,...,1628288,PER J. SKINNER MAIN BREAK AT LOCATION\nCall ta...,2017-01-22,2017-02-28,2017-02-28,2017,Turn off burst pipe,DIP,Ub,1


In [165]:
#clean data by converting all columns with strings to float/int or other type that works in ML 

#find all columns in dataframe where type = object 
df_types = pipes_df.select_dtypes(include=[np.object])

print(f'There are {len(df_types.dtypes)} columns containing strings')
print('--------------------------------------------------------------')
print(df_types.dtypes)

There are 23 columns containing strings
--------------------------------------------------------------
MOID                 object
MATERIAL             object
DESCRIPTION          object
PRESSURE_ZONE        object
LINING               object
PATTERN1             object
PATTERN2             object
PATTERN3             object
PATTERN4             object
PATTERN5             object
PATTERN6             object
PATTERN7             object
PATTERN8             object
PATTERN9             object
PATTERN10            object
WORK_ORDER_NUMBER    object
Comments             object
ReportDate           object
StartDateT           object
CompletedD           object
SrvcRqst_1           object
MATERIAL_1           object
MUSYM                object
dtype: object


In [166]:
#get dummies for columns where applicable--MATERIAL, DESCRIPTION, PRESSURE_ZONE
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['MATERIAL'], prefix='MATERIAL')], axis=1)
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['DESCRIPTION'], prefix='DESCRIPTION')], axis=1); 
pipes_df = pd.concat([pipes_df, pd.get_dummies(pipes_df['PRESSURE_ZONE'], prefix='P-ZONE')], axis=1); 
pipes_df.head()

,FID,MOID,DIAMETER,PIPE_LENGTH,MATERIAL,DESCRIPTION,YEAR_INSTALLED,PRESSURE_ZONE,LINING,LINE_YEAR,...,DESCRIPTION_Distribution,DESCRIPTION_Experimental Pipe not in CIP,DESCRIPTION_Facility Pipe,DESCRIPTION_Proposed Main: Adamsville,DESCRIPTION_Transmission,P-ZONE_Chatt 1020,P-ZONE_Hemp 1175,P-ZONE_Hemp 1175 (moved),P-ZONE_North 1225,P-ZONE_WTP
0,16420,P35881,8.0,861.090481,M.J.,Distribution,1952,Hemp 1175,,0,...,1,0,0,0,0,0,1,0,0,0
1,44465,P88079,8.0,749.511184,TYT,Distribution,1965,Hemp 1175,,0,...,1,0,0,0,0,0,1,0,0,0
2,23810,P53764,8.0,171.554040,M.J.,Distribution,1958,Hemp 1175,,0,...,1,0,0,0,0,0,1,0,0,0
3,52630,P400741,8.0,42.761097,M.J.,Distribution,1958,Hemp 1175,,0,...,1,0,0,0,0,0,1,0,0,0
4,10510,P20210,12.0,31.300481,DIP,Transmission,2006,Hemp 1175,,0,...,0,0,0,0,1,0,1,0,0,0


In [167]:
#drop original columns--MATERIAL, DESCRIPTION, PRESSURE_ZONE
pipes_df2 = pipes_df.drop(columns={ 'MATERIAL',
 'DESCRIPTION',
 'PRESSURE_ZONE',})
pipes_df2.head()

,FID,MOID,DIAMETER,PIPE_LENGTH,YEAR_INSTALLED,LINING,LINE_YEAR,INDEX_YR,ROUGHNESS,FLOW,...,DESCRIPTION_Distribution,DESCRIPTION_Experimental Pipe not in CIP,DESCRIPTION_Facility Pipe,DESCRIPTION_Proposed Main: Adamsville,DESCRIPTION_Transmission,P-ZONE_Chatt 1020,P-ZONE_Hemp 1175,P-ZONE_Hemp 1175 (moved),P-ZONE_North 1225,P-ZONE_WTP
0,16420,P35881,8.0,861.090481,1952,,0,1952,86,23.4711,...,1,0,0,0,0,0,1,0,0,0
1,44465,P88079,8.0,749.511184,1965,,0,1965,122,56.2002,...,1,0,0,0,0,0,1,0,0,0
2,23810,P53764,8.0,171.554040,1958,,0,1958,61,79.6832,...,1,0,0,0,0,0,1,0,0,0
3,52630,P400741,8.0,42.761097,1958,,0,1958,61,79.6832,...,1,0,0,0,0,0,1,0,0,0
4,10510,P20210,12.0,31.300481,2006,,0,2006,139,98.6106,...,0,0,0,0,1,0,1,0,0,0


In [168]:
# Now work on the'PATTERN' columns. There are 10 of them. 
#check if each pattern column only has 1 value
pipes_df2["PATTERN1"].value_counts()

RES    55937
          75
Name: PATTERN1, dtype: int64

In [169]:
pipes_df2["PATTERN2"].value_counts()

NON-RES    55937
              75
Name: PATTERN2, dtype: int64

In [170]:
pipes_df2["PATTERN3"].value_counts()

WHOLE    55937
            75
Name: PATTERN3, dtype: int64

In [171]:
pipes_df2["PATTERN4"].value_counts()

FLUSH    55937
            75
Name: PATTERN4, dtype: int64

In [172]:
pipes_df2["PATTERN5"].value_counts()

UFW    55937
          75
Name: PATTERN5, dtype: int64

In [173]:
pipes_df2["PATTERN6"].value_counts()

RES-DISTR    55937
                75
Name: PATTERN6, dtype: int64

In [174]:
pipes_df2["PATTERN7"].value_counts()

NON-RES-DISTR    55937
                    75
Name: PATTERN7, dtype: int64

In [175]:
pipes_df2["PATTERN8"].value_counts()

                56010
2030-CWTP-NS        2
Name: PATTERN8, dtype: int64

In [176]:
pipes_df2["PATTERN9"].value_counts()

     56012
Name: PATTERN9, dtype: int64

In [177]:
pipes_df2["PATTERN10"].value_counts()

     56012
Name: PATTERN10, dtype: int64

In [178]:
#Condense correlating PATTERN + DEMAND columns into 1 column
pipes_df2 = pipes_df2.rename(columns={"DEMAND1":"Demand_RES", "DEMAND2":"Demand_NON-RES","DEMAND3":"Demand_WHOLE","DEMAND4":"Demand_FLUSH","DEMAND5":"Demand_UFW",
                                     "DEMAND6":"Demand_RES-DISTR","DEMAND7":"Demand_NON RES-DISTR","DEMAND8":"Demand_2030-CWTP-NS"})

#Delete unneeded columns
#NOTE to Team: I didn’t see any data at all in the ‘DEMAND9’, ‘PATTERN9’, ‘DEMAND10’, ‘PATTERN10’ columns,so I deleted all of these columns
pipes_df3 = pipes_df2.drop(columns={"PATTERN1", "PATTERN2", "PATTERN3", "PATTERN4", "PATTERN5", "PATTERN6", "PATTERN7", "PATTERN8", "PATTERN9", "PATTERN10", "DEMAND9", "DEMAND10"})

In [179]:
pipes_df3.head()

,FID,MOID,DIAMETER,PIPE_LENGTH,YEAR_INSTALLED,LINING,LINE_YEAR,INDEX_YR,ROUGHNESS,FLOW,...,DESCRIPTION_Distribution,DESCRIPTION_Experimental Pipe not in CIP,DESCRIPTION_Facility Pipe,DESCRIPTION_Proposed Main: Adamsville,DESCRIPTION_Transmission,P-ZONE_Chatt 1020,P-ZONE_Hemp 1175,P-ZONE_Hemp 1175 (moved),P-ZONE_North 1225,P-ZONE_WTP
0,16420,P35881,8.0,861.090481,1952,,0,1952,86,23.4711,...,1,0,0,0,0,0,1,0,0,0
1,44465,P88079,8.0,749.511184,1965,,0,1965,122,56.2002,...,1,0,0,0,0,0,1,0,0,0
2,23810,P53764,8.0,171.554040,1958,,0,1958,61,79.6832,...,1,0,0,0,0,0,1,0,0,0
3,52630,P400741,8.0,42.761097,1958,,0,1958,61,79.6832,...,1,0,0,0,0,0,1,0,0,0
4,10510,P20210,12.0,31.300481,2006,,0,2006,139,98.6106,...,0,0,0,0,1,0,1,0,0,0


In [180]:
#ANUM'S STOPPING POINT
#These are the columns that still need to be converted to different types
df_types = pipes_df3.select_dtypes(include=[np.object])
df_types.dtypes

MOID                 object
LINING               object
WORK_ORDER_NUMBER    object
Comments             object
ReportDate           object
StartDateT           object
CompletedD           object
SrvcRqst_1           object
MATERIAL_1           object
MUSYM                object
dtype: object

In [181]:
#created new dataframe to change the datatype of ReportDate,StartDateT,CompletedD from object to datetime
df_date=pipes_df3

In [182]:
df_date["ReportDate"]=pd.to_datetime(df_date["ReportDate"], yearfirst=True, errors='coerce')
df_date["StartDateT"]=pd.to_datetime(df_date["StartDateT"],yearfirst=True, errors='coerce')
df_date["CompletedD"]=pd.to_datetime(df_date["CompletedD"],yearfirst=True, errors='coerce')

In [183]:
df_types11 = df_date.select_dtypes(include=[np.object])
df_types11.dtypes

MOID                 object
LINING               object
WORK_ORDER_NUMBER    object
Comments             object
SrvcRqst_1           object
MATERIAL_1           object
MUSYM                object
dtype: object

In [184]:
df_date["ReportDate"].head()

0   2014-10-25
1   2015-08-18
2   2015-09-11
3   2015-09-11
4   2017-01-22
Name: ReportDate, dtype: datetime64[ns]

In [185]:

df_date["CompletedD"].head()

0   2014-11-16
1   2015-08-18
2   2015-09-12
3   2015-09-12
4   2017-02-28
Name: CompletedD, dtype: datetime64[ns]

In [186]:
df_date["StartDateT"].head()

0   2014-11-16
1   2015-08-18
2   2015-09-12
3   2015-09-12
4   2017-02-28
Name: StartDateT, dtype: datetime64[ns]

In [187]:
pipes_df3.head()

,FID,MOID,DIAMETER,PIPE_LENGTH,YEAR_INSTALLED,LINING,LINE_YEAR,INDEX_YR,ROUGHNESS,FLOW,...,DESCRIPTION_Distribution,DESCRIPTION_Experimental Pipe not in CIP,DESCRIPTION_Facility Pipe,DESCRIPTION_Proposed Main: Adamsville,DESCRIPTION_Transmission,P-ZONE_Chatt 1020,P-ZONE_Hemp 1175,P-ZONE_Hemp 1175 (moved),P-ZONE_North 1225,P-ZONE_WTP
0,16420,P35881,8.0,861.090481,1952,,0,1952,86,23.4711,...,1,0,0,0,0,0,1,0,0,0
1,44465,P88079,8.0,749.511184,1965,,0,1965,122,56.2002,...,1,0,0,0,0,0,1,0,0,0
2,23810,P53764,8.0,171.554040,1958,,0,1958,61,79.6832,...,1,0,0,0,0,0,1,0,0,0
3,52630,P400741,8.0,42.761097,1958,,0,1958,61,79.6832,...,1,0,0,0,0,0,1,0,0,0
4,10510,P20210,12.0,31.300481,2006,,0,2006,139,98.6106,...,0,0,0,0,1,0,1,0,0,0


In [188]:
print([t for t in pipes_df3.dtypes])

[dtype('int64'), dtype('O'), dtype('float64'), dtype('float64'), dtype('int64'), dtype('O'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('int64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('O'), dtype('O'), dtype('<M8[ns]'), dtype('<M8[ns]'), dtype('<M8[ns]'), dtype('int64'), dtype('O'), dtype('O'), dtype('O'), dtype('int64'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('ui

In [189]:
teststring = "1/12/2018"

In [190]:
pipes_df3.iloc[1]['ReportDate'].year

2015

In [191]:
tempAgeList = []
for row in np.arange(len(pipes_df3['FID'])):
    try:
        value = int(pipes_df3.iloc[row]['ReportDate'].year) - pipes_df3.iloc[row]['YEAR_INSTALLED']
    except:
        value = 2018 - pipes_df3.iloc[row]['YEAR_INSTALLED']
    tempAgeList.append(value)

In [192]:
len(tempAgeList)

56012

In [193]:
pipes_df3['Age']=tempAgeList

In [194]:
pipes_df3['Age']

0         62
1         50
2         57
3         57
4         11
5         11
6         47
7         47
8         76
9         51
10        21
11       119
12        60
13        62
14        62
15        27
16        48
17        49
18        18
19        77
20        78
21        66
22       113
23        61
24        59
25        59
26        59
27        59
28        63
29        47
        ... 
55982     48
55983     48
55984     47
55985     47
55986     47
55987     46
55988     45
55989     45
55990     45
55991     45
55992     44
55993     44
55994     44
55995     43
55996     43
55997     43
55998     41
55999     40
56000     40
56001     40
56002     40
56003     39
56004     39
56005     39
56006     37
56007     24
56008     17
56009     17
56010     -6
56011     -7
Name: Age, Length: 56012, dtype: int64

In [195]:
pipes_df3['FAILURE']

0        1
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       1
14       1
15       1
16       1
17       1
18       1
19       1
20       1
21       1
22       1
23       1
24       1
25       1
26       1
27       1
28       1
29       1
        ..
55982    0
55983    0
55984    0
55985    0
55986    0
55987    0
55988    0
55989    0
55990    0
55991    0
55992    0
55993    0
55994    0
55995    0
55996    0
55997    0
55998    0
55999    0
56000    0
56001    0
56002    0
56003    0
56004    0
56005    0
56006    0
56007    0
56008    0
56009    0
56010    0
56011    0
Name: FAILURE, Length: 56012, dtype: int64

In [196]:
pipes_df3['MUSYM'].unique()

array(['Ub', 'ReD', 'UsE', 'CeC2', 'UrE', 'UfC2', 'AgC', 'PaD2', 'GaE',
       'UdC', 'UpC2', 'GaF', 'CaA', 'WbA', 'MdC2', 'UgE', 'CeB2', 'AgB',
       'UmC2', 'PgC2', 'UeE', 'UcC', 'CpA', 'Ua', 'ArE', 'RoE', 'W',
       'ReE', 'WcB', 'AaB', 'CrA', 'PaB2', 'Pt', 'AaC', 'LaD2', 'RoF',
       'WmD', 'MdB2', 'PaE2', 'EnC', 'HbC', 'AaA', 'BaA'], dtype=object)

In [197]:
inputColumns = ['DIAMETER', 'PIPE_LENGTH','ROUGHNESS', 'FLOW', 'VELOCITY', 'HEADLOSS',
       'HL1000', 'MAX_FLOW', 'MIN_FLOW', 'AVE_FLOW', 'MAX_VELOCITY',
       'MIN_VELOCITY', 'AVE_VELOCITY', 'MAX_HEADLOSS', 'MIN_HEADLOSS',
       'AVE_HEADLOSS', 'ELEVATION','RUN_ELEV', 'DEMAND',
       'HEAD', 'PRESSURE', 'MAX_HEAD', 'MIN_HEAD', 'AVE_HEAD', 'MAX_PRESS',
       'MIN_PRESS', 'AVE_PRESS', 'DIFF_PRESS','MATERIAL_AMTYT',
       'MATERIAL_CI', 'MATERIAL_CO', 'MATERIAL_COP', 'MATERIAL_CPP',
       'MATERIAL_Copper', 'MATERIAL_DEL', 'MATERIAL_DIP', 'MATERIAL_EARGEO',
       'MATERIAL_GP', 'MATERIAL_HDPE', 'MATERIAL_L.J.', 'MATERIAL_M.J.',
       'MATERIAL_MCW', 'MATERIAL_OTH', 'MATERIAL_PE', 'MATERIAL_PP',
       'MATERIAL_PVC', 'MATERIAL_R.L.J.', 'MATERIAL_RCP', 'MATERIAL_SP',
       'MATERIAL_ST', 'MATERIAL_T.D', 'MATERIAL_T.D.', 'MATERIAL_TTE',
       'MATERIAL_TYT', 'MATERIAL_UNK', 'MATERIAL_W.I.','Age']

In [198]:
import random

In [199]:
X_train, X_test, y_train, y_test = train_test_split(pipes_df3[inputColumns],pipes_df3['FAILURE'],stratify=pipes_df3['FAILURE'],test_size = 0.3, random_state=42)

In [200]:
model = LogisticRegression()

In [203]:
model.fit(X_train,y_train)
model.score(X_train,y_train)

0.9388135074474597

In [204]:
model.score(X_test,y_test)

0.9390621280647465

In [205]:
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)
X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)


In [206]:
X_train_false = X_train[y_train==0].sample(n=sum(y_train))
X_train_true = X_train[y_train==1].sample(n=sum(y_train))

In [207]:
X_train_sampled = X_train_true.append(X_train_false)
y_train_sampled = y_train.loc[X_train_sampled.index]

In [208]:
y_train_sampled.shape

(4776,)

In [209]:
X_train_minmax = X_minmax.transform(X_train_sampled)


In [210]:
model.fit(X_train_minmax,y_train_sampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [211]:
model.score(X_train_minmax,y_train_sampled)

0.7010050251256281

In [212]:
model.score(X_test_minmax,y_test)

0.6882885027374435

In [ ]:
model.score(X_test,y_test)

In [115]:
len(model.coef_[0])

57

In [116]:
len(inputColumns)

57

In [213]:
model.coef_[0]

array([-4.92427591e-01,  1.03410234e+01, -2.27007796e+00, -2.32105496e-01,
        7.20584408e-01,  3.22237239e-01,  4.73246594e-02, -7.89691824e-02,
        1.00848962e+00, -1.34226891e-03,  6.73203504e-01,  1.38676151e+00,
        9.18363688e-01,  7.84303637e-01,  2.12692609e-01,  6.42446418e-01,
       -2.68362829e+00,  7.67803076e-01,  1.61160405e-01, -1.85951096e-02,
       -2.03908161e+00,  8.38048437e-01,  1.15287238e+00, -7.02740011e-02,
        4.36702779e-01,  1.63357689e+00, -1.49646258e+00, -1.60236966e+00,
        0.00000000e+00,  3.17941822e-01,  2.02345732e+00,  0.00000000e+00,
       -1.42181861e+00,  0.00000000e+00,  2.31048016e-01, -1.32894364e-01,
        0.00000000e+00,  0.00000000e+00,  1.21044121e+00, -1.09325876e-01,
       -7.71640193e-02, -4.07494883e-01, -5.70078943e-01,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.17341076e+00,
        0.00000000e+00,  5.76277290e-01,  0.00000000e+00, -5.94942763e-01,
        4.70046336e-01,  

In [218]:
from sklearn.metrics import classification_report
print(classification_report(y_test, model.predict(X_test_minmax)))

             precision    recall  f1-score   support

          0       0.97      0.69      0.81     15780
          1       0.13      0.69      0.21      1024

avg / total       0.92      0.69      0.77     16804



In [219]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, model.predict(X_test_minmax)))

[[10861  4919]
 [  319   705]]


In [215]:
zipped = zip(model.coef_[0],inputColumns)

In [216]:
list(zipped)

[(-0.492427590827416, 'DIAMETER'),
 (10.341023386112086, 'PIPE_LENGTH'),
 (-2.2700779564600193, 'ROUGHNESS'),
 (-0.23210549623016039, 'FLOW'),
 (0.7205844075831638, 'VELOCITY'),
 (0.3222372393066158, 'HEADLOSS'),
 (0.04732465942520031, 'HL1000'),
 (-0.07896918241260457, 'MAX_FLOW'),
 (1.0084896217238928, 'MIN_FLOW'),
 (-0.0013422689083023797, 'AVE_FLOW'),
 (0.6732035040246637, 'MAX_VELOCITY'),
 (1.3867615082944595, 'MIN_VELOCITY'),
 (0.9183636881919685, 'AVE_VELOCITY'),
 (0.7843036367728268, 'MAX_HEADLOSS'),
 (0.21269260859789244, 'MIN_HEADLOSS'),
 (0.6424464181031179, 'AVE_HEADLOSS'),
 (-2.6836282910385107, 'ELEVATION'),
 (0.7678030762015707, 'RUN_ELEV'),
 (0.16116040542490295, 'DEMAND'),
 (-0.018595109638400945, 'HEAD'),
 (-2.039081606898418, 'PRESSURE'),
 (0.8380484374419508, 'MAX_HEAD'),
 (1.1528723844059898, 'MIN_HEAD'),
 (-0.07027400112885278, 'AVE_HEAD'),
 (0.4367027785505879, 'MAX_PRESS'),
 (1.6335768867908504, 'MIN_PRESS'),
 (-1.4964625765500115, 'AVE_PRESS'),
 (-1.60236965539

In [125]:
3/56

0.05357142857142857

In [ ]:
pipes_df3.iloc[pipes_df3]